In [1]:
import pandas as pd
import numpy as np
import nltk
import re
#importing stopwords is optional, in this case it decreased accuracy
#from nltk.corpus import stopwords
import itertools
import time

from nltk.stem.wordnet import WordNetLemmatizer 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\TIRTH
[nltk_data]     JOSHI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\TIRTH
[nltk_data]     JOSHI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\TIRTH
[nltk_data]     JOSHI\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

In [4]:
start_time = time.time()

In [5]:
train_data = pd.read_csv("C:/Users/TIRTH JOSHI/Desktop/Sigmoid ML/dataset/train.csv")

In [6]:
output_data = pd.read_csv("C:/Users/TIRTH JOSHI/Desktop/Sigmoid ML/dataset/test.csv")

In [7]:
emotions = train_data.columns.values[2:] 

In [8]:
emotions

array(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust', 'neutral'],
      dtype=object)

In [9]:
train_data

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,2017-En-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1,0
1,2017-En-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0,0
2,2017-En-21068,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0,0
3,2017-En-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0,0
4,2017-En-22195,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7719,2018-En-01993,@BadHombreNPS @SecretaryPerry If this didn't m...,1,0,1,0,0,0,0,0,0,0,0,0
7720,2018-En-01784,Excited to watch #stateoforigin tonight! Come ...,0,0,0,0,1,0,1,0,0,0,0,0
7721,2018-En-04047,"Blah blah blah Kyrie, IT, etc. @CJC9BOSS leavi...",1,0,1,0,0,0,0,0,1,0,0,0
7722,2018-En-03041,#ThingsIveLearned The wise #shepherd never tru...,0,0,0,0,0,0,0,0,0,0,0,1


In [10]:
def cleaning(text):
    txt = str(text)
    txt = re.sub(r"http\S+", "", txt)
    if len(txt) == 0:
        return 'no text'
    else:
        txt = txt.split()
        index = 0
        for j in range(len(txt)):
            if txt[j][0] == '@':
                index = j
        txt = np.delete(txt, index)
        if len(txt) == 0:
            return 'no text'
        else:
            words = txt[0]
            for k in range(len(txt)-1):
                words+= " " + txt[k+1]
            txt = words
            txt = re.sub(r'[^\w]', ' ', txt)
            if len(txt) == 0:
                return 'no text'
            else:
                txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt))
                txt = txt.replace("'", "")
                txt = nltk.tokenize.word_tokenize(txt)
                #data.content[i] = [w for w in data.content[i] if not w in stopset]
                for j in range(len(txt)):
                    txt[j] = lem.lemmatize(txt[j], "v")
                if len(txt) == 0:
                    return 'no text'
                else:
                    out = ""
                    for i in txt:
                        out += i + " "
                    return out

In [11]:
train_data['Tweet'] = train_data['Tweet'].map(lambda x: cleaning(x))

In [12]:
for emotion in emotions:
    lem = WordNetLemmatizer()
    temp = train_data[["Tweet",emotion]]
    emotion_data = temp.copy()
    x_train, x_test, y_train, y_test = train_test_split(emotion_data.Tweet, emotion_data[emotion], test_size=0.1, random_state=0)
    x_output = output_data["Tweet"]
    x_train = x_train.reset_index(drop = True)
    x_test = x_test.reset_index(drop = True)
    y_train = y_train.reset_index(drop = True)
    y_test = y_test.reset_index(drop = True)
    x_output = x_output.reset_index(drop = True)
    vectorizer = TfidfVectorizer(min_df=3, max_df=0.9)
    train_vectors = vectorizer.fit_transform(x_train)
    test_vectors = vectorizer.transform(x_test)
    output_vectors = vectorizer.transform(x_output)
    model = svm.SVC(kernel='rbf',gamma='scale',probability = True,class_weight = 'balanced', degree = 8) 
    model.fit(train_vectors, y_train) 
    predicted_sentiment = model.predict(test_vectors)
    output_sentiment = model.predict(output_vectors)
    print(emotion)
    print(classification_report(y_test, predicted_sentiment))
    predicted_sentiments = []
    for s in range(len(output_sentiment)):
        predicted_sentiments.append(output_sentiment[s])
    output_data[emotion] = predicted_sentiments
#     prediction_df = pd.DataFrame({'Content':x_test,  emotion: predicted_sentiment})
# prediction_df.to_csv('emotion_recognizer_svm.csv', index = False)

anger
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       471
           1       0.76      0.71      0.74       302

    accuracy                           0.80       773
   macro avg       0.79      0.78      0.79       773
weighted avg       0.80      0.80      0.80       773

anticipation
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       670
           1       0.27      0.14      0.18       103

    accuracy                           0.84       773
   macro avg       0.57      0.54      0.54       773
weighted avg       0.80      0.84      0.81       773

disgust
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       484
           1       0.67      0.67      0.67       289

    accuracy                           0.75       773
   macro avg       0.74      0.74      0.74       773
weighted avg       0.75      0.75      0.75    

In [25]:
from pathlib import Path  
filepath = Path('C:/Users/TIRTH JOSHI/Desktop/Sigmoid ML/dataset/output.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
output_data.to_csv(filepath)

In [14]:
df2 = train_data[["Tweet","anticipation"]]

In [15]:
df2

,Tweet,anticipation
0,be a down payment on a problem you may never h...,1
1,you decide to do make sure it make you happy,0
2,it also help that the majority of NFL coach be...,0
3,the challenge so that you can literally even f...,0
4,roommate it s okay that we can t spell because...,0
...,...,...
7719,BadHombreNPS If this didn t make me so angry I...,0
7720,to watch stateoforigin tonight Come on NSW ori...,0
7721,Blah blah blah Kyrie IT etc leave Boston be th...,0
7722,The wise shepherd never trust his flock to a s...,0


In [16]:
x_train, x_test, y_train, y_test = train_test_split(df2.Tweet, df2.anticipation, test_size=0.25, random_state=0)

In [17]:
x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)

In [18]:
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [19]:
vectorizer = TfidfVectorizer(min_df=3, max_df=0.9)

In [20]:
train_vectors = vectorizer.fit_transform(x_train)
test_vectors = vectorizer.transform(x_test)

In [21]:
model = svm.SVC(kernel='rbf', degree=8) 
model.fit(train_vectors, y_train) 
predicted_sentiment = model.predict(test_vectors)

In [22]:
print(classification_report(y_test, predicted_sentiment))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1669
           1       0.40      0.01      0.01       262

    accuracy                           0.86      1931
   macro avg       0.63      0.50      0.47      1931
weighted avg       0.80      0.86      0.80      1931



In [23]:
predicted_sentiments = []
for s in range(len(predicted_sentiment)):
    predicted_sentiments.append(predicted_sentiment[s])

In [24]:
prediction_df = pd.DataFrame({'Content':x_test, 'Emotion_predicted':predicted_sentiment, 'Emotion_actual': y_test})
prediction_df.to_csv('emotion_recognizer_svm.csv', index = False)